In [1]:
!nvidia-smi

Thu Aug 11 00:31:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# ignoring warnings
import warnings
warnings.simplefilter('ignore')

In [3]:
!pip install -q efficientnet_pytorch

In [4]:
!pip install -U albumentations

In [5]:
!pip install opencv-python-headless==4.2.0.34

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 31.3 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.4.60
    Uninstalling opencv-python-headless-4.5.4.60:
      Successfully uninstalled opencv-python-headless-4.5.4.60


In [6]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 2.3 MB/s eta 0:00:00


In [7]:
import warnings
warnings.simplefilter('ignore')

import timm
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torchvision import transforms
import torch
import torchvision
import torchvision.models as models
from efficientnet_pytorch import EfficientNet
from torch.optim.lr_scheduler import MultiStepLR
from torch.optim.lr_scheduler import OneCycleLR
import pandas as pd 
import numpy as np
import sklearn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, roc_auc_score
from tqdm import tqdm_notebook as tqdm 
from sklearn.model_selection import train_test_split

#Keras 
from keras.utils.np_utils import to_categorical

In [8]:
import random
SEED_VAL  = 15
# Set the seed value all over the place to make this reproducible.
def seed_all(SEED):
    random.seed(SEED_VAL)
    np.random.seed(SEED_VAL)
    torch.manual_seed(SEED_VAL)
    torch.cuda.manual_seed_all(SEED_VAL)
    os.environ['PYTHONHASHSEED'] = str(SEED_VAL)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [9]:
import cv2
def default_image_loader(path):
    return Image.open(path).convert('RGB')

def cv_reader(path):
  img = cv2.imread(path)
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  return img

class ImageDataset(Dataset):
    def __init__(self, data_path, df, transform=None,mode='train'):
        self.df = df
        self.loader = cv_reader
        self.transform = transform
        self.dir = data_path
        self.mode = mode

    def __getitem__(self, index):
        image_name = self.df.image_name[index]
        image = self.loader(os.path.join(self.dir, image_name))
        if self.transform is not None:
          image = self.transform(image=image)
          image=image["image"]

          output = {
              'image': image,
          }

        if self.mode == 'train':
            label = self.df.loc[index, 'target']
            output.update({
              'label': torch.tensor(label, dtype=torch.long)
          })

        return output          
        
    
    def __len__(self):
        return self.df.shape[0]

In [10]:
import torch.nn.functional as F
class Net(nn.Module):
    def __init__(self, name):
        super(Net, self).__init__()
        self.name  = name
        if name == 'b0':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b0')
          self.arch._fc = nn.Linear(in_features=1280, out_features=3, bias=True)
        elif name == 'b1':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b1')
          self.arch._fc = nn.Linear(in_features=1280, out_features=3, bias=True)
        elif name == 'b2':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b2')
          self.arch._fc = nn.Linear(in_features=1408, out_features=3, bias=True)
        elif name =='b3':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b3')
          self.arch._fc = nn.Linear(in_features=1536, out_features=3, bias=True)
        elif name =='b4':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b4')
          self.arch._fc = nn.Linear(in_features=1792, out_features=3, bias=True)
        elif name =='b5':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b5')
          self.arch._fc = nn.Linear(in_features=2048, out_features=3, bias=True)
        elif name =='b6':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b6')
          self.arch._fc = nn.Linear(in_features=2304, out_features=3, bias=True)
        elif name =='b7':
          self.arch =  EfficientNet.from_pretrained('efficientnet-b7')
          self.arch._fc = nn.Linear(in_features=2560, out_features=3, bias=True)
        elif name == 'densenet121':
          self.arch = models.densenet121(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,3,bias=True)
        elif name == 'densenet169':
          self.arch = models.densenet169(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,3,bias=True)
        elif name == 'densenet201':
          self.arch = models.densenet201(pretrained=True)
          num_ftrs = self.arch.classifier.in_features
          self.arch.classifier = nn.Linear(num_ftrs,3,bias=True)
        elif name == 'resnet50':
          self.arch = models.resnet50(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,3,bias=True)
        elif name == 'resnet101':
          self.arch = models.resnet101(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,3,bias=True)
        elif name == 'resnet152':
          self.arch = models.resnet152(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,3,bias=True)
        elif name == 'resnet34':
          self.arch = models.resnet34(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,3,bias=True)
        elif name == 'resnext101':
          self.arch = models.resnext101_32x8d(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          self.arch.fc = nn.Linear(num_ftrs,3,bias=True)
        elif name == 'resnext50':
          self.arch = models.resnext50_32x4d(pretrained=True)
          num_ftrs = self.arch.fc.in_features
          
          self.arch.fc = nn.Linear(num_ftrs,3,bias=True)
        elif name =='rexnetv1':
            model = rexnetv1.ReXNetV1(width_mult=1.0)
            model.output.conv2D = nn.Conv2d(1280, 3, kernel_size=(1, 1), stride=(1, 1))
      
        elif name == 'swin_base_patch4_window12_384_in22k':
          self.arch = timm.create_model("swin_base_patch4_window12_384_in22k", pretrained=True,  num_classes=3)
        
        elif name == 'swin_large_patch4_window12_384_in22k':
          self.arch = timm.create_model("swin_large_patch4_window12_384_in22k", pretrained=True,  num_classes=3)

#         elif name == 'ig_resnext101_32x8d':
#           self.arch = timm.create_model("ig_resnext101_32x8d", pretrained=True,  num_classes=3)
      
    def forward(self, x):
        x = self.arch(x)
        return x

In [11]:
class AverageMeter():
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [13]:
def loss_fn(outputs,targets):
  criterion = nn.CrossEntropyLoss()
  loss = criterion(outputs,targets)
  return loss

In [14]:
def train_fn(train_data_loader, model, optimizer, device, scheduler=None):
  avg_acc=0
  model.to(device)
  model.train()
  losses = AverageMeter()
  tk0 = tqdm(train_data_loader, total=len(train_data_loader))
  tot_loss = 0
  for bi,d in enumerate(tk0):
    images = d['image']
    labels = d['label']

    #send them to device 
    images = images.to(device, dtype=torch.float)
    labels = labels.to(device)
    optimizer.zero_grad()
    if BOOL_CROP:
      bs, ncrops, c, h, w = images.size()
      outputs = model(images.view(-1, c, h, w)) # fuse batch size and ncrops
      outputs = outputs.view(bs, ncrops, -1).mean(1) # avg over crops
    
    else:
      outputs  = model(images)

    loss = loss_fn(outputs, labels)
    outputs = F.softmax(outputs, dim=1)
#     tot_loss += loss
    tot_loss = tot_loss + loss.item()
    avg_acc += (labels == outputs.argmax(1)).float().mean()
#     losses.update(loss.item(), labels.size(0))
    
    loss.backward()
    optimizer.step()
    


    if scheduler is not None:
      scheduler.step()

  loglosss = tot_loss/len(train_data_loader)
  avg_acc_nump = avg_acc.cpu().detach().numpy()
  acc_avg = avg_acc_nump/len(train_data_loader)
  
  print("Training loss for this epoch: ", loglosss)
  return loglosss

In [15]:
def eval_fn(valid_data_loader,model,device):
  model.eval()
  losses = AverageMeter()
  tot_loss = 0
  final_outputs = []
  final_targets = []
  with torch.no_grad():
    for bi,d in enumerate(valid_data_loader):

      images = d['image']
      labels = d['label']

      #send them to device 
      images = images.to(device,dtype=torch.float)
      labels = labels.to(device)
      if BOOL_CROP:
        bs, ncrops, c, h, w = images.size()
        outputs = model(images.view(-1, c, h, w)) # fuse batch size and ncrops
        outputs = outputs.view(bs, ncrops, -1).mean(1) # avg over crops
      else:
        outputs  = model(images)

      loss = loss_fn(outputs,labels)
      outputs = F.softmax(outputs, dim=1)
#       tot_loss += loss
      tot_loss = tot_loss + loss.item()
      losses.update(loss.item(), labels.size(0))

      
      final_outputs.append(outputs.cpu().detach().numpy())
      final_targets.append(labels.cpu().numpy())


    final_targets = np.concatenate(final_targets)
    final_outputs = np.concatenate(final_outputs)
    loglosss  = tot_loss/len(valid_data_loader)
    print("Validation loss for this epoch: ",loglosss)
  return loglosss

In [16]:
import albumentations as A
from albumentations.pytorch.transforms import ToTensor
from albumentations.pytorch.transforms import ToTensorV2

input_shape = [128,256]

def get_transforms():
    train_transform = A.Compose([
                              A.Resize(384, 384, p=1),
                              A.RandomResizedCrop(height=384, width=384, p=1),
                              A.Flip(p=0.5),
                              A.HorizontalFlip(p=0.5),
                              A.VerticalFlip(p=0.5),
                              A.Rotate(15),
                              A.ShiftScaleRotate(p=0.5),
                              A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
                              A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),  
#                               A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, always_apply=False, p=0.5)
                              A.CoarseDropout(p=0.5),
                              A.Cutout(p=0.5),
                              A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                              ToTensorV2(),
                            ])
    val_transform = A.Compose([
                              A.Resize(height=384, width=384, p=1),
                              A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                              ToTensorV2(),
                            ])
    return train_transform,val_transform

In [17]:
train = pd.read_csv('../input/rice-class/Train.csv')
train = train.rename(columns = {'Image_id':'image_name','Label':'target'})

In [18]:
#identify partial string to look for
discard = ["_rgn"]

#drop rows that contain the partial string "_rgn" in the conference column
train = train[~train.image_name.str.contains('|'.join(discard))]

In [19]:
train.reset_index(drop=True, inplace=True)

In [20]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train['target'] = le.fit_transform(train['target'])

In [21]:
%%time
from sklearn.model_selection import StratifiedKFold, KFold
NAME = 'swin_large_patch4_window12_384_in22k'
EPOCHS = 15
TRAIN_BATH_SIZE = 4
BOOL_CROP = False
skf = StratifiedKFold(n_splits=5)
all_loss_scores = []
def run_folds():
  seed_all(SEED_VAL)
  train_transform,val_transform = get_transforms()
  DF = pd.DataFrame()
  for i,(train_index,val_index) in enumerate(skf.split(train, y=train.target)):
    print(f"#########################  Fold {i+1}/{skf.n_splits}  #########################")
    train_df , valid_df = train.iloc[train_index,:],train.iloc[val_index,:]
    train_df = train_df.reset_index()
    valid_df = valid_df.reset_index()
    train_dataset = ImageDataset('../input/microsoft-rice-disease-classification-challenge', train_df,train_transform)
    valid_dataset = ImageDataset('../input/microsoft-rice-disease-classification-challenge',valid_df,val_transform)
    train_data_loader = DataLoader(dataset=train_dataset,shuffle=True,batch_size=TRAIN_BATH_SIZE)
    valid_data_loader = DataLoader(dataset=valid_dataset,shuffle=False, batch_size=4)
    device = torch.device("cuda")
    model = Net(NAME)
    model.to(device)

    num_train_steps = int(len(train_df) /TRAIN_BATH_SIZE * EPOCHS)
    optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
    scheduler  = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=1, factor=0.7,min_lr=1e-6,verbose=True)
    best_loss = 3.0
    
    for epoch in range(EPOCHS):
      print("----------------EPOCH "+str(epoch+1)+"---------------------")
      loss_train = train_fn(train_data_loader, model, optimizer, device,scheduler=None)
      loss_val = eval_fn(valid_data_loader ,model, device)
      scheduler.step(loss_val)
      if loss_val<best_loss:
        best_loss = loss_val
        torch.save(model.state_dict(),f"best_model_{i}") 
    print(f'best VAL_LOSS for fold {i+1}: ',best_loss)
    DF = DF.append({'FOLD':1+i,'BEST_LOG_LOSS':best_loss},ignore_index=True)
    all_loss_scores.append(best_loss)
  print(f"MEAN LogLoss: {np.mean(all_loss_scores)}")
  return DF

CPU times: user 31 µs, sys: 0 ns, total: 31 µs
Wall time: 34.6 µs


In [22]:
%%time
kfold_results = run_folds()

#########################  Fold 1/5  #########################


Downloading: "https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_large_patch4_window12_384_22k.pth" to /root/.cache/torch/hub/checkpoints/swin_large_patch4_window12_384_22k.pth


----------------EPOCH 1---------------------


  0%|          | 0/534 [00:00<?, ?it/s]

Training loss for this epoch:  0.5323752230416969
Validation loss for this epoch:  0.20777816665984356
----------------EPOCH 2---------------------


...

Training loss for this epoch:  0.0617093335586531
Validation loss for this epoch:  0.12122257376009528
----------------EPOCH 14---------------------


  0%|          | 0/534 [00:00<?, ?it/s]

Training loss for this epoch:  0.06299141929948048
Validation loss for this epoch:  0.0879206209707711
----------------EPOCH 15---------------------


  0%|          | 0/534 [00:00<?, ?it/s]

Training loss for this epoch:  0.05586305642843777
Validation loss for this epoch:  0.08538746630340179
best VAL_LOSS for fold 5:  0.08538746630340179
MEAN LogLoss: 0.07116472182151926
CPU times: user 7h 51s, sys: 3min 11s, total: 7h 4min 3s
Wall time: 7h 23s


In [23]:
kfold_results

,FOLD,BEST_LOG_LOSS
0,1.0,0.043493
1,2.0,0.054272
2,3.0,0.102341
3,4.0,0.070331
4,5.0,0.085387


In [24]:
test_transform = A.Compose([        
                              A.Resize(height=384, width=384,p=1),
                              A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
                              ToTensorV2()])
test = pd.read_csv('../input/rice-class/SampleSubmission.csv')
test.rename(columns={'Image_id':'image_name'},inplace=True)
test_dataset = ImageDataset('../input/microsoft-rice-disease-classification-challenge',test, test_transform, mode='test')

In [25]:
def predict(bs=4):
  test_ds = ImageDataset('./Images',test, test_transform,mode='test')
  testloader = DataLoader(dataset=test_dataset,shuffle=False,batch_size=bs)

  predictions_labels = []
  predictions_proba = []

  out = None

  for data in tqdm(testloader):
    x = data['image'].to(device, dtype=torch.float)

    for i in range(skf.n_splits):
      if i == 0: out = F.softmax(MODELS[i](x), dim=-1)
      else: out += F.softmax(MODELS[i](x), dim=-1)

    out /= skf.n_splits
    out_labels = out.argmax(1).cpu().detach().numpy()
    out_probas = out.cpu().detach().numpy()

    predictions_labels += out_labels.tolist()
    predictions_proba += out_probas.tolist()

  return predictions_labels ,predictions_proba

In [26]:
MODELS = []
for i in range(skf.n_splits):
  MODELS.append(Net(name=NAME))
  MODELS[i].to(device)
  MODELS[i].load_state_dict(torch.load(f'best_model_{i}'))
  MODELS[i].eval()

In [27]:
torch.cuda.empty_cache()

In [28]:
predictions_labels, predictions_proba = predict(bs=1)

  0%|          | 0/1145 [00:00<?, ?it/s]

In [29]:
sub1 = pd.read_csv("../input/rice-class/SampleSubmission.csv")
rice = sub1.columns[1:]
rice

Index(['blast', 'brown', 'healthy'], dtype='object')

In [30]:
submission = pd.DataFrame()
submission['Image_id'] = sub1['Image_id']
for i, label in enumerate(rice):
  submission[label] = 0
for i, label in enumerate(rice):
  submission.loc[:, label] = np.array(predictions_proba)[:,i]

In [31]:
submission.head()

,Image_id,blast,brown,healthy
0,id_00vl5wvxq3.jpg,0.999998,2.079400e-06,1.350421e-07
1,id_01hu05mtch.jpg,0.000345,9.996501e-01,4.915459e-06
2,id_030ln10ewn.jpg,0.872913,1.269678e-01,1.196852e-04
3,id_03z57m8xht.jpg,1.000000,1.568779e-07,1.062301e-08
4,id_04ngep1w4b.jpg,0.999626,3.162101e-04,5.739113e-05


In [32]:
submission.to_csv('new_scheduler0.0582.csv', index=False)